In [1]:
from functools import partial
from pathlib import Path
import time
from zipfile import ZipFile

import napari
from napari.utils import nbscreenshot

from shapely.geometry import GeometryCollection
from shapely.geometry import Point

from scipy.ndimage import affine_transform
from skimage.transform import AffineTransform

import dask.array as da
from dask.array.core import normalize_chunks
from dask_image.ndfilters import gaussian_filter

#from spacem_lib.io.writer import write_ome_zarr

from utils.download_sample import download_from_dropbox
from utils.coordinate_utils import extract_coordinates, normalize_coords_to_pixel
from utils.imutils import *
from utils.shapely_and_napari_utils import *
from utils.chunks import get_chunk_coordinates, get_rect_from_chunk_slice, tile_chunk_intersections
from fuse.fuse import fuse_func

In [2]:
%gui qt

In [3]:
time.sleep(1)

In [4]:
if not Path("Stiching_Example_Dataset").exists():
    # Download and extract sample data
    example_dataset="https://www.dropbox.com/s/rhggl54sbpx9llj/Stiching_Example_Dataset.zip?dl=0"
    print("Downloading sample file from dropbox")
    download_from_dropbox(example_dataset, "_sample.zip")
    print("Extracting sample file")
    ZipFile("_sample.zip").extractall()

In [5]:
coords = extract_coordinates("Stiching_Example_Dataset/out.txt")
coords

,name,X,Y,Z,um/px
0,1_0000,3582.7,4164.4,1686.65,0.64
1,1_0001,-12396.6,11719.3,1686.63,0.64
2,1_0002,-13583.0,11726.0,1686.63,0.64
3,1_0003,-14769.3,11732.8,1686.63,0.64
4,1_0004,-15955.6,11739.5,1686.63,0.64
...,...,...,...,...,...
67,1_0067,-16003.1,3370.3,1686.63,0.64
68,1_0068,-14816.8,3363.5,1686.63,0.64
69,1_0069,-13630.5,3356.8,1686.63,0.64
70,1_0070,-12444.2,3350.1,1686.63,0.64


In [6]:
files = sorted(list(Path("Stiching_Example_Dataset").glob("*.jpg")))

In [7]:
def subsample(arr, factor: int=2):
    return arr[::factor, ::factor]

In [8]:
normalized_coords=normalize_coords_to_pixel(coords).to_numpy()

In [9]:
factor = 16
_subsample = partial(subsample, factor=factor)

load_transforms = [crop_black_border,
                    _subsample,]
                   # transpose]
_load_image = partial(load_image, transforms=load_transforms)

In [10]:
images = list(map(_load_image, files))

In [11]:
eight_transforms = ((), 
                    (transpose,), 
                    (np.fliplr,), 
                    (np.flipud,), 
                    (np.flipud, transpose,), 
                    (np.fliplr, transpose,), 
                    (np.flipud, np.fliplr,), 
                    (np.flipud, np.fliplr, transpose,),)

In [12]:
chunk_size = (4096,4096)
_fuse_func=partial(fuse_func, 
                   imload_func=None,
                   dtype=np.uint8)
previews =[]
for transform_chain in eight_transforms:
    mosaics_shape_images = get_mosaics_bboxes_transforms(images,transform_chain,normalized_coords,factor)
    _tmp = get_chunk_slices_and_shapes(chunk_size=chunk_size, array_shape=mosaics_shape_images['stitched_shape'])
    chunk_slices = _tmp['chunk_slices']
    chunks_shapely = _tmp['chunks_shapely']
    chunk_tiles = tile_chunk_intersections(mosaics_shape_images['mosaic_shifted'],
                                       mosaics_shape_images['transformed_images'],
                                       mosaics_shape_images['transforms'],
                                       chunks_shapely, 
                                       chunk_slices)
    target_array = da.map_blocks(func=_fuse_func,
                             chunks=normalize_chunks(chunk_size, mosaics_shape_images['stitched_shape']), 
                             input_tile_info=chunk_tiles,
                             dtype=np.uint8)
    previews.append(target_array.compute())

Processing chunk at (0, 0)
Processing chunk at (0, 0)
Processing chunk at (0, 0)
Processing chunk at (0, 0)
Processing chunk at (0, 0)
Processing chunk at (0, 0)
Processing chunk at (0, 0)
Processing chunk at (0, 0)


In [13]:
v = napari.Viewer()

2021-07-02 13:55:07,615 [INFO] WRITING LOG OUTPUT TO /home/hilsenst/.cellpose/run.log


In [14]:
[v.add_image(p) for p in previews]

[<Image layer 'p' at 0x7f2c42d6ea10>,
 <Image layer 'p [1]' at 0x7f2c42f1b2d0>,
 <Image layer 'p [2]' at 0x7f2c42eb51d0>,
 <Image layer 'p [3]' at 0x7f2c434a1710>,
 <Image layer 'p [4]' at 0x7f2c4326ecd0>,
 <Image layer 'p [5]' at 0x7f2c432b8dd0>,
 <Image layer 'p [6]' at 0x7f2c43120510>,
 <Image layer 'p [7]' at 0x7f2c42aef150>]